# Frequency

In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

feature_pos = [[0,8],[8,12],[12,28],[28,44],[44,52],[52,64],[64,80],[80,92],[92,102]] 
df_train = pd.read_csv('train_kaggle.csv')
df_test = pd.read_csv('sample_solution.csv')
Y = df_train['Label'].values

## Save Frequency Features
Skip this if you have saved the features

In [43]:
import numpy as np

dataframes = np.load('allData.npy', allow_pickle = True)
dataframes.shape

def load_test_dataframe(id):
    test_data = np.load("test/test/{}.npy".format(id))
    return pd.DataFrame(data=test_data)

testdatas = []
for id in df_test['Id']:
    dfi = load_test_dataframe(id)
    testdatas.append(dfi.values)

In [44]:
feat = dataframes[0].shape[1]
# feat = 12

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

def one_hot_encoding(encoder, dfs, col):
    if encoder == None:
        data = set(dfs[0][:,col])
        for i in range(1,len(dfs)):
            df = dfs[i]
            data |= set(df[:,col])
        encoder = OneHotEncoder(handle_unknown='ignore')
        encoder = encoder.fit([[str(e)] for e in data])
    encodedData = []
    for df in dfs:
        mat = np.sum(encoder.transform([[str(e)] for e in df[:,col]]), axis=0)
        mat = np.squeeze(np.asarray(mat))
        encodedData.append(mat)
    encodedData = np.stack(encodedData)    
    return encoder, encodedData

def one_hot_encoding_feature(encoders, dfs, feature):
    if len(encoders) == 0:
        encoder, data = one_hot_encoding(None, dfs, feature_pos[feature][0])
        encoders.append(encoder)    
        for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
            encoder, dataCol = one_hot_encoding(None, dfs, col)
            encoders.append(encoder)
            data = np.concatenate((data, dataCol), axis=1)
    else:
        encoder, data = one_hot_encoding(encoders[0], testdatas, feature_pos[feature][0])
        for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
            i = col - feature_pos[feature][0]
            encoder, dataCol = one_hot_encoding(encoders[i], testdatas, col)
            data = np.concatenate((data, dataCol), axis=1)
    return encoders, data

In [ ]:
import gc

# Give up last feature
for feat in range(len(feature_pos) - 1):
    print(feat)
    encoders, train = one_hot_encoding_feature([], dataframes, feat)
    encoders, test = one_hot_encoding_feature(encoders, testdatas, feat)
    np.save('freq/' + str(feat) + '_train.npy', train)
    np.save('freq/' + str(feat) + '_test.npy', test)
    gc.collect()

In [45]:
from tqdm import tqdm

def handle_integers(dfs, col):
    data = []
    for df in dfs:
        sumCol = np.sum(df[:,col], axis=0).reshape(1, -1)
        meanCol = np.mean(df[:, col], axis=0).reshape(1, -1)
        # stdCol = np.mean(df[:, col]).reshape(1, -1)
        dataCol = np.concatenate([sumCol, meanCol], axis=1)
        # dataCol = sumCol
        # print(dataCol.shape)
        data.append(dataCol)
    data = np.concatenate(data, axis=0)    
    # print(data.shape)
    return data

# Extract based on correlations
select_int_col = [True, False, False, True, True, False, False, False, False, True]
def handle_integers_feature(dfs, feature):
    if feature != 8:
        return None
    data = handle_integers(dfs, feature_pos[feature][0])
    for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
        i = col - feature_pos[feature][0]
        if select_int_col[i] == False:
            continue
        dataCol = handle_integers(dfs, col)
        data = np.concatenate((data, dataCol), axis=1)
    return data

train = handle_integers_feature(dataframes, 8)
test = handle_integers_feature(testdatas, 8)
np.save('freq/8_train.npy', train)
np.save('freq/8_test.npy', test)

100%|█████████████████████████████████████| 9/9 [00:00<00:00, 12.92it/s]


## Load Frequency Features

In [3]:
import numpy as np
feature_selected = [True,True,True,True,True,True,True,True,True] 

def load_freq(feat, postfix):
    data = np.load("freq/{}_{}.npy".format(feat, postfix))
    return data

def load_test_freq(feat):
    return load_freq(feat, 'test')

def load_train_freq(feat):
    return load_freq(feat, 'train')

In [4]:
def select_features(feature_selected):
    XTrain = []
    XTest = []
    for feat in range(len(feature_pos)):
        if feature_selected[feat] == False:
            continue
        train = load_train_freq(feat)
        test = load_test_freq(feat)
        XTrain.append(train)
        XTest.append(test)
    XTrain = np.concatenate(XTrain, axis=1)
    XTest = np.concatenate(XTest, axis=1)
    return XTrain, XTest

def all_not_selected(feature_selected):
    for i in range(len(feature_pos)):
        if feature_selected[i] == True:
            return False
    return True

In [11]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn import metrics
import gc

scores = [0]
f0s = [0]
f1s = [0]
f2s = [0]
f3s = [0]
f4s = [0]
f5s = [0]
f6s = [0]
f7s = [0]
score = (0, 0, 0, 0, 0, 0, 0, 0, 0)
cnt = 0
for f0 in range(2):
    feature_selected[0] = bool(f0)
    for f1 in range(2):
        feature_selected[1] = bool(f1)
        for f2 in range(2):
            feature_selected[2] = bool(f2)
            for f3 in range(2):
                feature_selected[3] = bool(f3)
                for f4 in range(2):
                    feature_selected[4] = bool(f4)
                    for f5 in range(2):
                        feature_selected[5] = bool(f5)
                        for f6 in range(2):
                            feature_selected[6] = bool(f6)
                            for f7 in range(2):
                                cnt += 1
                                feature_selected[7] = bool(f7)
                                if all_not_selected(feature_selected) == True:
                                    continue
                                if cnt % 10 == 0:
                                    print(cnt)
                                XTrain, XTest = select_features(feature_selected)
                                del XTest
                                XTrain, X_valid, y_train, y_valid = train_test_split(XTrain, Y, test_size=0.2, random_state=42)
                                gc.collect()
                                model = LGBMClassifier()
                                model.fit(XTrain, y_train)
                                pred = model.predict_proba(X_valid)[:,1]
                                fpr, tpr, thresholds = metrics.roc_curve(y_valid, pred)
                                auc_score = metrics.auc(fpr, tpr)
                                scores.append(auc_score)
                                f0s.append(f0)
                                f1s.append(f1)
                                f2s.append(f2)
                                f3s.append(f3)
                                f4s.append(f4)
                                f5s.append(f5)
                                f6s.append(f6)
                                f7s.append(f7)
                                gc.collect()
                                if score[0] < auc_score:
                                    score = (auc_score, f0, f1, f2, f3, f4, f5, f6, f7)

KeyboardInterrupt: 

In [17]:
auc_csv = pd.DataFrame({'f0': f0s, 'f1': f1s, 'f2': f2s, 'f3': f3s, 'f4': f4s, 'f5': f5s, 'f6': f6s, 'f7': f7s, 'auc': scores}).sort_values(by=['auc'])
auc_csv.to_csv('freq_auc.csv')
auc_csv

,f0,f1,f2,f3,f4,f5,f6,f7,auc
0,0,0,0,0,0,0,0,0,0.000000
2,0,0,0,0,0,0,1,0,0.882728
1,0,0,0,0,0,0,0,1,0.889711
3,0,0,0,0,0,0,1,1,0.895471
5,0,0,0,0,0,1,0,1,0.980764
4,0,0,0,0,0,1,0,0,0.980828
6,0,0,0,0,0,1,1,0,0.981217
7,0,0,0,0,0,1,1,1,0.982000
11,0,0,0,0,1,0,1,1,0.982529
18,0,0,0,1,0,0,1,0,0.982658


In [15]:
print(score)

(0.992238129804707, 1, 1, 1, 1, 0, 0, 0, 0)


In [16]:
for i in range(8):
    feature_selected[i] = bool(score[i + 1])

XTrain, XTest = select_features(feature_selected)

## Try Others

In [5]:
feature_selected = [True,True,True,False,True,False,False,False,True] 
XTrain, XTest = select_features(feature_selected)

## Train models

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(XTrain, Y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(14929, 69153)
(3733, 69153)
(14929,)
(3733,)


In [7]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train, y_train)
valid_set = lgb.Dataset(X_valid, y_valid)

from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(XTrain, Y)
modelSelection = SelectFromModel(clf, prefit=True, max_features=8000)
XTrain = modelSelection.transform(XTrain)
XTrain.shape

XTest = modelSelection.transform(XTest)
XTest.shape

In [8]:
from hyperopt import hp
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import tpe
import lightgbm as lgb
from hyperopt import STATUS_OK

ITER = 50
STOP_ROUND = 5

# Create the dataset
def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    params['num_leaves'] = int(params['num_leaves'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    bst = lgb.train(params, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
  
    # Extract the best score
    best_score = bst.best_score['valid_0']['auc']
    
    # Loss must be minimized
    loss = np.log(1 - best_score)
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Define the search space
space = {
    'boosting_type': 'dart',
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'metric': 'auc'
}

# Trials object to track progress
bayes_trials = Trials()

MAX_EVALS = 500

# Optimize
bestDict = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials,
            rstate=np.random.RandomState(1024) # fixing random state for the reproducibility
           )
bestDict

[1]	valid_0's auc: 0.948204                                             
  0%|                             | 0/500 [00:07<?, ?it/s, best loss: ?]

C:\Users\jw.chan\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



[2]	valid_0's auc: 0.961317                                             
[3]	valid_0's auc: 0.963639                                             
[4]	valid_0's auc: 0.965598                                             
[5]	valid_0's auc: 0.967489                                             
[6]	valid_0's auc: 0.969007                                             
[7]	valid_0's auc: 0.970116                                             
[8]	valid_0's auc: 0.970397                                             
[9]	valid_0's auc: 0.971513                                             
[10]	valid_0's auc: 0.972906                                            
[11]	valid_0's auc: 0.973522                                            
[12]	valid_0's auc: 0.973316                                            
[13]	valid_0's auc: 0.974345                                            
[14]	valid_0's auc: 0.975113                                            
[15]	valid_0's auc: 0.97548                        

[14]	valid_0's auc: 0.975031                                            
[15]	valid_0's auc: 0.97542                                             
[16]	valid_0's auc: 0.975982                                            
[17]	valid_0's auc: 0.976231                                            
[18]	valid_0's auc: 0.976525                                            
[19]	valid_0's auc: 0.977117                                            
[20]	valid_0's auc: 0.977595                                            
[21]	valid_0's auc: 0.977566                                            
[22]	valid_0's auc: 0.978131                                            
[23]	valid_0's auc: 0.978816                                            
[24]	valid_0's auc: 0.979113                                            
[25]	valid_0's auc: 0.979208                                            
[26]	valid_0's auc: 0.979733                                            
[27]	valid_0's auc: 0.98014                        

[26]	valid_0's auc: 0.9816                                              
[27]	valid_0's auc: 0.98177                                             
[28]	valid_0's auc: 0.981852                                            
[29]	valid_0's auc: 0.982075                                            
[30]	valid_0's auc: 0.98214                                             
[31]	valid_0's auc: 0.982231                                            
[32]	valid_0's auc: 0.982312                                            
[33]	valid_0's auc: 0.982551                                            
[34]	valid_0's auc: 0.982592                                            
[35]	valid_0's auc: 0.982517                                            
[36]	valid_0's auc: 0.98255                                             
[37]	valid_0's auc: 0.982622                                            
[38]	valid_0's auc: 0.982837                                            
[39]	valid_0's auc: 0.983089                       

...

[42]	valid_0's auc: 0.989982                                            
[43]	valid_0's auc: 0.990023                                            
[44]	valid_0's auc: 0.990006                                            
[45]	valid_0's auc: 0.990041                                            
[46]	valid_0's auc: 0.99007                                             
[47]	valid_0's auc: 0.990594                                            
[48]	valid_0's auc: 0.990458                                            
[49]	valid_0's auc: 0.990488                                            
[50]	valid_0's auc: 0.990512                                            
[1]	valid_0's auc: 0.954129                                             
[2]	valid_0's auc: 0.966818                                             
[3]	valid_0's auc: 0.968592                                             
[4]	valid_0's auc: 0.969544                                             
[5]	valid_0's auc: 0.971812                        

[4]	valid_0's auc: 0.974402                                             
[5]	valid_0's auc: 0.976214                                             
[6]	valid_0's auc: 0.977686                                             
[7]	valid_0's auc: 0.979099                                             
[8]	valid_0's auc: 0.979411                                             
[9]	valid_0's auc: 0.979856                                             
[10]	valid_0's auc: 0.980269                                            
[11]	valid_0's auc: 0.980852                                            
[12]	valid_0's auc: 0.980635                                            
[13]	valid_0's auc: 0.981177                                            
[14]	valid_0's auc: 0.98121                                             
[15]	valid_0's auc: 0.98172                                             
[16]	valid_0's auc: 0.982094                                            
[17]	valid_0's auc: 0.982275                       

[16]	valid_0's auc: 0.989373                                            
[17]	valid_0's auc: 0.989757                                            
[18]	valid_0's auc: 0.989898                                            
[19]	valid_0's auc: 0.989809                                            
[20]	valid_0's auc: 0.989965                                            
[21]	valid_0's auc: 0.990069                                            
[22]	valid_0's auc: 0.9902                                              
[23]	valid_0's auc: 0.990051                                            
[24]	valid_0's auc: 0.99049                                             
[25]	valid_0's auc: 0.990701                                            
[26]	valid_0's auc: 0.99046                                             
[27]	valid_0's auc: 0.990594                                            
[28]	valid_0's auc: 0.990523                                            
[29]	valid_0's auc: 0.990508                       

[28]	valid_0's auc: 0.98663                                             
[29]	valid_0's auc: 0.986807                                            
[30]	valid_0's auc: 0.986856                                            
[31]	valid_0's auc: 0.986917                                            
[32]	valid_0's auc: 0.98721                                             
[33]	valid_0's auc: 0.987349                                            
[34]	valid_0's auc: 0.987642                                            
[35]	valid_0's auc: 0.987652                                            
[36]	valid_0's auc: 0.987686                                            
[37]	valid_0's auc: 0.987976                                            
[38]	valid_0's auc: 0.98812                                             
[39]	valid_0's auc: 0.98828                                             
[40]	valid_0's auc: 0.988311                                            
[41]	valid_0's auc: 0.988334                       

{'colsample_by_tree': 0.6823940952011491,
 'learning_rate': 0.17789597000387453,
 'min_child_samples': 20.0,
 'num_leaves': 148.0,
 'reg_alpha': 0.5958632522649805,
 'reg_lambda': 0.7055329967022398,
 'subsample_for_bin': 140000.0}

In [9]:
from hyperopt import space_eval
best = space_eval(space, bestDict)
best

{'boosting_type': 'dart',
 'colsample_bytree': 0.6823940952011491,
 'learning_rate': 0.17789597000387453,
 'metric': 'auc',
 'min_child_samples': 20.0,
 'num_leaves': 148.0,
 'reg_alpha': 0.5958632522649805,
 'reg_lambda': 0.7055329967022398,
 'subsample_for_bin': 140000.0}

In [10]:
best['num_leaves'] = int(best['num_leaves'])
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['min_child_samples'] = int(best['min_child_samples'])
best['metric'] = 'auc'
lgbModel = lgb.train(best, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
lgbModel.save_model('model.txt', num_iteration=lgbModel.best_iteration)

# Extract the best score
best_score = lgbModel.best_score['valid_0']['auc']

# Loss must be minimized
loss = np.log(1 - best_score)
print(loss)
YTest = lgbModel.predict(XTest)

[1]	valid_0's auc: 0.974851
[2]	valid_0's auc: 0.982623
[3]	valid_0's auc: 0.984262
[4]	valid_0's auc: 0.985274
[5]	valid_0's auc: 0.985875
[6]	valid_0's auc: 0.987547
[7]	valid_0's auc: 0.988307
[8]	valid_0's auc: 0.988389
[9]	valid_0's auc: 0.988949
[10]	valid_0's auc: 0.988765
[11]	valid_0's auc: 0.989188
[12]	valid_0's auc: 0.989083
[13]	valid_0's auc: 0.989453
[14]	valid_0's auc: 0.989767
[15]	valid_0's auc: 0.989907
[16]	valid_0's auc: 0.990151
[17]	valid_0's auc: 0.990634
[18]	valid_0's auc: 0.990843
[19]	valid_0's auc: 0.991029
[20]	valid_0's auc: 0.991217
[21]	valid_0's auc: 0.991273
[22]	valid_0's auc: 0.991298
[23]	valid_0's auc: 0.991407
[24]	valid_0's auc: 0.991324
[25]	valid_0's auc: 0.991536
[26]	valid_0's auc: 0.991564
[27]	valid_0's auc: 0.991666
[28]	valid_0's auc: 0.991656
[29]	valid_0's auc: 0.991671
[30]	valid_0's auc: 0.991782
[31]	valid_0's auc: 0.991724
[32]	valid_0's auc: 0.991716
[33]	valid_0's auc: 0.991803
[34]	valid_0's auc: 0.991899
[35]	valid_0's auc: 0.9

In [11]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(boosting_type=best['boosting_type'],
                        num_leaves=best['num_leaves'],
                        learning_rate=best['learning_rate'],
                        subsample_for_bin=best['subsample_for_bin'],
                        min_child_samples=best['min_child_samples'],
                        reg_alpha=best['reg_alpha'],
                        reg_lambda=best['reg_lambda'],
                        colsample_bytree=best['colsample_bytree'])
model.fit(XTrain, Y)
probs = model.predict_proba(XTest, num_iteration=lgbModel.best_iteration)

In [12]:
YTest = probs[:, 0]

In [13]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.000998
1,1,0.009700
2,2,0.998428
3,3,0.015736
4,4,0.789423
5,5,0.950752
6,6,0.998758
7,7,0.903690
8,8,0.000692
9,9,0.000978


In [14]:
df_test.to_csv('test.csv', index=False)